TODOS:
- DataAugmentation (inkl. Rotation der Velocities)

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import math

import h5py
import os

import rb_equivariant_cnn as conv
import rb_equivariant_gcnn as gconv
import rb_equivariant_se2ncnn as dn_conv

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

2024-08-15 02:50:30.650484: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-15 02:50:30.682059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 02:50:31.147328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=16384)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2024-08-15 02:50:32.449992: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-08-15 02:50:32.451611: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16384 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:01:00.0, compute capability: 8.0


# Setup

In [3]:
RB_CHANNELS = 4
HORIZONTAL_SIZE = 48
HEIGHT = 32

BATCH_SIZE = 64

SIMULATION_NAME = '48_48_32_2000_0.71_0.01_0.3_1000.2'

# Data

In [4]:
sim_file = os.path.join('data', f'{SIMULATION_NAME}.h5')

with h5py.File(sim_file, 'r') as hf:
    N_TRAIN = hf['train'].attrs['N']
    N_TEST = hf['test'].attrs['N']
    
TRAIN_BATCHES = math.ceil(N_TRAIN/BATCH_SIZE)
TEST_BATCHES = math.ceil(N_TEST/BATCH_SIZE)

class generator:
    def __init__(self, filename, dataset):
        self.filename = filename
        self.dataset = dataset


    def __call__(self):
        with h5py.File(self.filename, 'r') as hf:
            while True:
                for snap in hf[self.dataset]:
                    yield snap, snap

train_dataset = tf.data.Dataset.from_generator(
     generator(sim_file, 'train'),
     output_signature=(
         tf.TensorSpec(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS), dtype=tf.float64),
         tf.TensorSpec(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS), dtype=tf.float64)))

test_dataset = tf.data.Dataset.from_generator(
     generator(sim_file, 'test'),
     output_signature=(
         tf.TensorSpec(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS), dtype=tf.float64),
         tf.TensorSpec(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS), dtype=tf.float64)))

# dataset = dataset.shuffle(10, reshuffle_each_iteration=True)
train_dataset = train_dataset.batch(BATCH_SIZE, False)
test_dataset = test_dataset.batch(BATCH_SIZE, False)

# Data Augmentation

In [4]:

#TODO  Rotate and Flip Vectors
class RandomRot(keras.layers.RandomRotation):
    def call(self, inputs: tf.Tensor, *args, **kwargs) -> tf.Tensor:
        in_shape = tf.shape(inputs)
        inputs = tf.reshape(inputs, in_shape[:-2]+[np.prod(in_shape[-2:])])
        outputs = super().call(inputs, *args, **kwargs)
        return tf.reshape(outputs, in_shape)
    
class RandomFlip(keras.layers.RandomFlip):
    def call(self, inputs: tf.Tensor, *args, **kwargs) -> tf.Tensor:
        in_shape = tf.shape(inputs)
        inputs = tf.reshape(inputs, in_shape[:-2]+[np.prod(in_shape[-2:])])
        outputs = super().call(inputs, *args, **kwargs)
        return tf.reshape(outputs, in_shape)

# 3D Rayleigh-Bénard Convolution
- Equivariant to horizontal translations
- __No vertical parameter sharing__
- Height dependend bias
- Supports horizontal wrap and same padding
    - Wrap makes sense when using peridoc boundary conditions for Rayleigh-Bénard
    - Attention: This may destroy exact rotation equivariance in our experiments (nevertheless WRAP will be preferable in practice)
- Also supports vertical same padding
- Supports stride (including vertical stride)
- Uses 2D convolutions under the hood

In [6]:
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            # Data Augmentation
            RandomRot(factor=1, fill_mode='wrap', value_range=(0,1)),
            RandomFlip(mode='horizontal_and_vertical'),
            
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv1'),
            conv.BatchNorm(name='BatchNorm1'),
            keras.layers.Activation('relu', name='NonLinearity1'),
            
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv2'),
            conv.BatchNorm(name='BatchNorm2'),
            keras.layers.Activation('relu', name='NonLinearity2'),
            
            conv.SpatialPooling(ksize=(2,2,2), strides=(2,2,2), pooling_type='MAX'),
            
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv3'),
            conv.BatchNorm(name='BatchNorm3'),
            keras.layers.Activation('relu', name='NonLinearity3'),
            
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2,2,2), name='Conv4'),
            conv.BatchNorm(name='BatchNorm4'),
            keras.layers.Activation('relu', name='NonLinearity4'),
        ])

# output shape: batch_size, width, depth, height, channels
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_rot (RandomRot)          │ (64, 96, 96, 64, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (64, 96, 96, 64, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 (RB3D_Conv)               │ (64, 49, 49, 33, 4)    │        23,892 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm1 (BatchNorm)          │ (64, 49, 49, 33, 4)    │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity1 (Activation)      │ (64, 49, 49, 33, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (64, 49, 49, 33, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 (RB3D_Conv)               │ (64, 25, 25, 17, 4)    │        12,308 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm2 (BatchNorm)          │ (64, 25, 25, 17, 4)    │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity2 (Activation)      │ (64, 25, 25, 17, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling (SpatialPooling) │ (64, 12, 12, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (64, 12, 12, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv3 (RB3D_Conv)               │ (64, 7, 7, 5, 4)       │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm3 (BatchNorm)          │ (64, 7, 7, 5, 4)       │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity3 (Activation)      │ (64, 7, 7, 5, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (64, 7, 7, 5, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv4 (RB3D_Conv)               │ (64, 4, 4, 3, 4)       │         2,172 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm4 (BatchNorm)          │ (64, 4, 4, 3, 4)       │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ NonLinearity4 (Activation)      │ (64, 4, 4, 3, 4)       │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,920 (167.66 KB)

 Trainable params: 42,456 (165.84 KB)

 Non-trainable params: 464 (1.81 KB)

# 3D Rayleigh-Bénard $D_4$ Group Equivariant Convolution
- Equivariant to all symmetries of 3D Rayleigh-Bénard:
    - __90° rotations around a vertical axis__
    - __reflections through a vertical plane__
    - __horizontal translations__

In [7]:
G = 'D4' # 'C4' for rotations or 'D4' for rotations and reflections
model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            gconv.RB3D_G_Conv('Z2', G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'Lift_{G}_Conv1'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv2'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv3'),
            gconv.RB3D_G_Conv(G,    G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(2, 2, 2), 
                              name=f'{G}_Conv4'),
        ])

# output shape: batch_size, width, depth, transformations, height, channels
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (64, 96, 96, 1, 64, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Lift_D4_Conv1 (RB3D_G_Conv)     │ (64, 49, 49, 8, 33, 4) │        23,892 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv2 (RB3D_G_Conv)          │ (64, 25, 25, 8, 17, 4) │        97,988 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv3 (RB3D_G_Conv)          │ (64, 13, 13, 8, 9, 4)  │        51,876 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ D4_Conv4 (RB3D_G_Conv)          │ (64, 7, 7, 8, 5, 4)    │        28,820 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,576 (791.31 KB)

 Trainable params: 202,576 (791.31 KB)

 Non-trainable params: 0 (0.00 B)

# 3D Rayleigh-Bénard $D_N$ Group Equivariant Convolution
- Equivariant to all symmetries of 3D Rayleigh-Bénard:
    - __arbitrary discrete__ rotations around a vertical axis
    - reflections through a vertical plane
    - horizontal translations

In [8]:
ORIENTATIONS = 8

model = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            dn_conv.RB3D_LiftDN_Conv(orientations=ORIENTATIONS, h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP',
                                     v_padding='SAME', strides=(2, 2, 2), name='Lift_DN_Conv1'),
            dn_conv.RB3D_DN_Conv(h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', 
                                 v_padding='SAME', strides=(2, 2, 2), name='DN_Conv2'),
            dn_conv.RB3D_DN_Conv(h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', 
                                 v_padding='SAME', strides=(2, 2, 2), name='DN_Conv3'),
            dn_conv.RB3D_DN_Conv(h_ksize=5, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', 
                                 v_padding='SAME', strides=(2, 2, 2), name='DN_Conv4'),
        ])

# output shape: batch_size, width, depth, transformations, height, channels
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Lift_DN_Conv1                   │ (64, 49, 49, 16, 33,   │        66,132 │
│ (RB3D_LiftDN_Conv)              │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DN_Conv2 (RB3D_DN_Conv)         │ (64, 25, 25, 16, 17,   │       544,068 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DN_Conv3 (RB3D_DN_Conv)         │ (64, 13, 13, 16, 9, 4) │       288,036 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DN_Conv4 (RB3D_DN_Conv)         │ (64, 7, 7, 16, 5, 4)   │       160,020 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,058,256 (4.04 MB)

 Trainable params: 1,058,256 (4.04 MB)

 Non-trainable params: 0 (0.00 B)

# Autoencoder

#### Standard 3D Convolutional Autoencoder

In [5]:
ae = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            ###############
            #   Encoder   #
            ###############
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.MaxPooling3D(pool_size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.MaxPooling3D(pool_size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.MaxPooling3D(pool_size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.MaxPooling3D(pool_size=(2,2,2)),
            
            ###############
            #   Decoder   #
            ###############
            keras.layers.UpSampling3D(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.UpSampling3D(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.UpSampling3D(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.UpSampling3D(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Conv3D(kernel_size=(3,3,5), filters=RB_CHANNELS, padding='same', kernel_regularizer=keras.regularizers.L2(0.01)),
        ])

# output shape: batch_size, width, depth, height, channels
ae.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (64, 48, 48, 32, 4)    │           724 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (64, 48, 48, 32, 4)    │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (64, 48, 48, 32, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (64, 24, 24, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (64, 24, 24, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (64, 24, 24, 16, 4)    │           724 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (64, 24, 24, 16, 4)    │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (64, 24, 24, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (64, 12, 12, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (64, 12, 12, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (64, 12, 12, 8, 4)     │           724 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (64, 12, 12, 8, 4)     │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (64, 12, 12, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (64, 6, 6, 4, 4)       │           724 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (64, 6, 6, 4, 4)       │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (64, 3, 3, 2, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling3d (UpSampling3D)    │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (64, 6, 6, 4, 4)       │           724 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (64, 6, 6, 4, 4)       │            1

 Total params: 5,904 (23.06 KB)

 Trainable params: 5,848 (22.84 KB)

 Non-trainable params: 56 (224.00 B)

In [6]:
ae.compile(
    loss=tf.keras.losses.MeanSquaredError,
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
)

hist = ae.fit(train_dataset, steps_per_epoch=TRAIN_BATCHES, validation_data=test_dataset, validation_steps=TEST_BATCHES, epochs=100)

Epoch 1/100


I0000 00:00:1723677235.303914   88675 service.cc:145] XLA service 0x7f0258009910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723677235.303989   88675 service.cc:153]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-08-15 01:13:55.396663: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-15 01:13:55.598984: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/42 ━━━━━━━━━━━━━━━━━━━━ 7:10 10s/step - loss: 0.5997

I0000 00:00:1723677244.064649   88675 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


42/42 ━━━━━━━━━━━━━━━━━━━━ 23s 300ms/step - loss: 0.3453 - val_loss: 0.5966
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 314ms/step - loss: 0.3307 - val_loss: 0.5636
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 12s 295ms/step - loss: 0.2960 - val_loss: 0.4639
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 11s 262ms/step - loss: 0.2251 - val_loss: 0.4004
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 12s 293ms/step - loss: 0.1703 - val_loss: 0.4046
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 12s 282ms/step - loss: 0.1337 - val_loss: 0.3972
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 11s 279ms/step - loss: 0.1104 - val_loss: 0.3948
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 11s 278ms/step - loss: 0.0973 - val_loss: 0.4000
Epoch 9/100
 2/42 ━━━━━━━━━━━━━━━━━━━━ 16s 423ms/step - loss: 0.1719

KeyboardInterrupt: 

#### Height dependend Convolutional Autoencoder

In [8]:
h_ae = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            ###############
            #   Encoder   #
            ###############
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
            conv.BatchNorm(),
            keras.layers.Activation('relu'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
            conv.BatchNorm(),
            keras.layers.Activation('relu'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
            conv.BatchNorm(),
            keras.layers.Activation('relu'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
            conv.BatchNorm(),
            keras.layers.Activation('relu'),
            conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            
            ###############
            #   Decoder   #
            ###############
            conv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
            conv.BatchNorm(),
            keras.layers.Activation('relu'),
            conv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
            conv.BatchNorm(),
            keras.layers.Activation('relu'),
            conv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
            conv.BatchNorm(),
            keras.layers.Activation('relu'),
            conv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            conv.RB3D_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                           filter_regularizer=keras.regularizers.L2(0.01)),
        ])

# output shape: batch_size, width, depth, height, channels
h_ae.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ RB3D_Conv9 (RB3D_Conv)          │ (64, 48, 48, 32, 4)    │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm1 (BatchNorm)          │ (64, 48, 48, 32, 4)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (64, 48, 48, 32, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling5                 │ (64, 24, 24, 16, 4)    │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (64, 24, 24, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_Conv10 (RB3D_Conv)         │ (64, 24, 24, 16, 4)    │        11,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm2 (BatchNorm)          │ (64, 24, 24, 16, 4)    │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (64, 24, 24, 16, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling6                 │ (64, 12, 12, 8, 4)     │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (64, 12, 12, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_Conv11 (RB3D_Conv)         │ (64, 12, 12, 8, 4)     │         5,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm3 (BatchNorm)          │ (64, 12, 12, 8, 4)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (64, 12, 12, 8, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling7                 │ (64, 6, 6, 4, 4)       │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_Conv12 (RB3D_Conv)         │ (64, 6, 6, 4, 4)       │         2,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm4 (BatchNorm)          │ (64, 6, 6, 4, 4)       │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling8                 │ (64, 3, 3, 2, 4)       │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling3d_8 (UpSampling3D)  │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (64, 6, 6, 4, 4)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_Conv13 (RB3D_Conv)         │ (64, 6, 6, 4, 4)       │         2,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm5 (BatchNorm)          │ (64, 6, 6, 4, 4)       │            6

 Total params: 88,288 (344.88 KB)

 Trainable params: 87,584 (342.12 KB)

 Non-trainable params: 704 (2.75 KB)

In [9]:
h_ae.compile(
    loss=tf.keras.losses.MeanSquaredError,
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
)

hist = h_ae.fit(train_dataset, steps_per_epoch=TRAIN_BATCHES, validation_data=test_dataset, validation_steps=TEST_BATCHES, epochs=100)

Epoch 1/100


I0000 00:00:1723679090.605354   98967 service.cc:145] XLA service 0x7f355c00b260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723679090.605423   98967 service.cc:153]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-08-15 01:44:50.784806: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-15 01:44:51.308987: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 2/42 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.7101  

I0000 00:00:1723679103.718369   98967 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


34/42 ━━━━━━━━━━━━━━━━━━━━ 1s 236ms/step - loss: 0.4756

KeyboardInterrupt: 

### $D_4$ Group Equivariant Convolutional Autoencoder

In [5]:
G = 'D4' # 'C4' for rotations or 'D4' for rotations and reflections

d4_ae = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            # add transformation dimension
            keras.layers.Reshape((HORIZONTAL_SIZE, HORIZONTAL_SIZE, 1, HEIGHT, RB_CHANNELS)), 
            
            ###############
            #   Encoder   #
            ###############
            gconv.RB3D_G_Conv('Z2', G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            
            gconv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            
            ###############
            #   Decoder   #
            ###############
            gconv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            
            gconv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            
            gconv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            
            gconv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            gconv.RB3D_G_Conv(G, G, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                              filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.TransformationPooling(tf.reduce_mean, keepdims=False)
        ])

# output shape: batch_size, width, depth, height, channels
d4_ae.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape (Reshape)               │ (64, 48, 48, 1, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv1 (RB3D_G_Conv)      │ (64, 48, 48, 8, 32, 4) │        23,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm1 (BatchNorm)          │ (64, 48, 48, 8, 32, 4) │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (64, 48, 48, 8, 32, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling1                 │ (64, 24, 24, 8, 16, 4) │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (64, 24, 24, 8, 16, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv2 (RB3D_G_Conv)      │ (64, 24, 24, 8, 16, 4) │        92,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm2 (BatchNorm)          │ (64, 24, 24, 8, 16, 4) │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (64, 24, 24, 8, 16, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling2                 │ (64, 12, 12, 8, 8, 4)  │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (64, 12, 12, 8, 8, 4)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv3 (RB3D_G_Conv)      │ (64, 12, 12, 8, 8, 4)  │        46,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm3 (BatchNorm)          │ (64, 12, 12, 8, 8, 4)  │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (64, 12, 12, 8, 8, 4)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling3                 │ (64, 6, 6, 8, 4, 4)    │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (64, 6, 6, 8, 4, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv4 (RB3D_G_Conv)      │ (64, 6, 6, 8, 4, 4)    │        23,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm4 (BatchNorm)          │ (64, 6, 6, 8, 4, 4)    │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (64, 6, 6, 8, 4, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling4                 │ (64, 3, 3, 8, 2, 4)    │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ UpSampling1 (UpSampling)        │ (64, 6, 6, 8, 4, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (64, 6, 6, 8, 4, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv5 (RB3D_G_Conv)      │ (64, 6, 6, 8, 4, 4)    │        23,05

 Total params: 531,808 (2.03 MB)

 Trainable params: 531,104 (2.03 MB)

 Non-trainable params: 704 (2.75 KB)

In [6]:
d4_ae.compile(
    loss=tf.keras.losses.MeanSquaredError,
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
)

hist = d4_ae.fit(train_dataset, steps_per_epoch=TRAIN_BATCHES, validation_data=test_dataset, validation_steps=TEST_BATCHES, epochs=100)

Epoch 1/100


I0000 00:00:1723679957.488550  104125 service.cc:145] XLA service 0x7f110401c0b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723679957.488631  104125 service.cc:153]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-08-15 01:59:17.721194: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-15 01:59:18.286564: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-08-15 01:59:23.898404: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,1024,48,48]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,144,50,50]{3,2,1,0}, f32[1024,144,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_co

42/42 ━━━━━━━━━━━━━━━━━━━━ 163s 403ms/step - loss: 0.2047 - val_loss: 0.5262
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 15s 350ms/step - loss: 0.2691 - val_loss: 0.5341
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 344ms/step - loss: 0.2732 - val_loss: 0.3797
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 322ms/step - loss: 0.1845 - val_loss: 0.5462
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 315ms/step - loss: 0.2611 - val_loss: 0.5243
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 328ms/step - loss: 0.2790 - val_loss: 0.5118
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 314ms/step - loss: 0.2523 - val_loss: 0.5440
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 315ms/step - loss: 0.2777 - val_loss: 0.4435
Epoch 9/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 317ms/step - loss: 0.2322 - val_loss: 0.5150
Epoch 10/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 14s 326ms/step - loss: 0.2574 - val_loss: 0.5184
Epoch 11/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 13s 313ms/step - loss: 0.2468 - val_loss: 0.4688
Epoch 12/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1

### $D_N$ Group Equivariant Convolutional Autoencoder

In [18]:
ORIENTATIONS = 8

dn_ae = keras.Sequential([
            keras.layers.InputLayer(shape=(HORIZONTAL_SIZE, HORIZONTAL_SIZE, HEIGHT, RB_CHANNELS),
                                    batch_size=BATCH_SIZE),
            
            ###############
            #   Encoder   #
            ###############
            dn_conv.RB3D_LiftDN_Conv(orientations=ORIENTATIONS, h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                     filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                 filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                 filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            keras.layers.Dropout(rate=0.2),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                 filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            dn_conv.SpatialPooling(ksize=(2,2,2), pooling_type='MAX', strides=(2,2,2), padding='VALID'),
            
            ###############
            #   Decoder   #
            ###############
            dn_conv.UpSampling(size=(2,2,2), name='UpSampling1'),
            keras.layers.Dropout(rate=0.2),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                 filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            dn_conv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                 filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            dn_conv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                 filter_regularizer=keras.regularizers.L2(0.01)),
            gconv.BatchNorm(),
            keras.layers.Activation('relu'),
            dn_conv.UpSampling(size=(2,2,2)),
            keras.layers.Dropout(rate=0.2),
            dn_conv.RB3D_DN_Conv(h_ksize=3, v_ksize=5, channels=RB_CHANNELS, h_padding='WRAP', v_padding='SAME', strides=(1,1,1),
                                 filter_regularizer=keras.regularizers.L2(0.01)),
            dn_conv.TransformationPooling(tf.reduce_mean, keepdims=False)
        ])

# output shape: batch_size, width, depth, transformations, height, channels
dn_ae.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ RB3D_G_Conv7 (RB3D_LiftDN_Conv) │ (64, 48, 48, 16, 32,   │        23,168 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm29 (BatchNorm)         │ (64, 48, 48, 16, 32,   │           512 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_28 (Activation)      │ (64, 48, 48, 16, 32,   │             0 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling25                │ (64, 24, 24, 16, 16,   │             0 │
│ (SpatialPooling)                │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (64, 24, 24, 16, 16,   │             0 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv43 (RB3D_DN_Conv)    │ (64, 24, 24, 16, 16,   │       184,384 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm30 (BatchNorm)         │ (64, 24, 24, 16, 16,   │           256 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_29 (Activation)      │ (64, 24, 24, 16, 16,   │             0 │
│                                 │ 4)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling26                │ (64, 12, 12, 16, 8, 4) │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (64, 12, 12, 16, 8, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv44 (RB3D_DN_Conv)    │ (64, 12, 12, 16, 8, 4) │        92,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm31 (BatchNorm)         │ (64, 12, 12, 16, 8, 4) │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_30 (Activation)      │ (64, 12, 12, 16, 8, 4) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling27                │ (64, 6, 6, 16, 4, 4)   │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (64, 6, 6, 16, 4, 4)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RB3D_G_Conv45 (RB3D_DN_Conv)    │ (64, 6, 6, 16, 4, 4)   │        46,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNorm32 (BatchNorm)         │ (64, 6, 6, 16, 4, 4)   │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_31 (Activation)      │ (64, 6, 6, 16, 4, 4)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SpatialPooling28                │ (64, 3, 3, 16, 2, 4)   │             0 │
│ (SpatialPooling)                │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,038,688 (3.96 MB)

 Trainable params: 1,037,984 (3.96 MB)

 Non-trainable params: 704 (2.75 KB)

In [19]:
dn_ae.compile(
    loss=tf.keras.losses.MeanSquaredError,
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
)

hist = dn_ae.fit(train_dataset, steps_per_epoch=TRAIN_BATCHES, validation_data=test_dataset, validation_steps=TEST_BATCHES, epochs=100)

Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 50s 842ms/step - loss: 0.7314 - val_loss: 0.8166
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 771ms/step - loss: 0.5262 - val_loss: 0.6416
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 773ms/step - loss: 0.4119 - val_loss: 0.5820
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 770ms/step - loss: 0.3651 - val_loss: 0.5664
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 769ms/step - loss: 0.3394 - val_loss: 0.5471
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 771ms/step - loss: 0.3221 - val_loss: 0.5449
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 771ms/step - loss: 0.3106 - val_loss: 0.5128
Epoch 8/100
15/42 ━━━━━━━━━━━━━━━━━━━━ 19s 721ms/step - loss: 0.4255